# Importeren van de nodige libraries

In [ ]:
from gymnasium import spaces
import pygame
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import random
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import BaseCallback
import time

## Aanmaken van de Snake omgeving

In [ ]:
class SnakeEnv(gym.Env):
    """
    De class SnakeEnv is een Gymnasium Environment die de mogelijkheid geeft om
    het spel Snake te benaderen met Reïnforcement Learning.

    Parameters:
    -----------
        render_mode : str (of None)
            De render mode voor de pygame
    """
    metadata = {"render_modes": ["human"]}

    def __init__(self, render_mode=None):
        """
        De initializer van de class. Hier worden meerdere standaard attributen aangemaakt
        """
        super(SnakeEnv, self).__init__()
        self.grid_size = 20
        self.cell_size = 30
        self.action_space = spaces.Discrete(4)  # 0=UP, 1=RIGHT, 2=DOWN, 3=LEFT
        self.observation_space = spaces.Box(
            low=0, high=2,
            shape=(3, self.grid_size, self.grid_size),
            dtype=np.int32
)
        self.render_mode = render_mode
        self.window = None
        self.clock = None
        self.obs_buffer = []    

    def reset(self, seed=None, options=None):
        """"
        De reset method van de environment. Zorgt dat de game weer van het begin kan beginnen
        """
        # Aanmaken van de snake en appel
        self.snake = [(5, 5), (5, 5), (5, 5)]
        self.direction = 1
        self.food = self._place_food()
        self.done = False

        # Beginnen met renderen van de Pygame bij render_mode == "human"
        if self.render_mode == "human":
            self._init_render()
        return self._get_obs(), {}

    def _place_food(self):
        """
        Een method die in staat is om voedsel op het speelveld te plaatsen.
        """
        while True:
            food = (random.randint(0, 19), random.randint(0, 19))

            # Voorkomen dat het voedsel niet in de slang wordt geplaatst
            if food not in self.snake:
                return food

    def _get_obs(self):
        """
        Een method die de observaties van het spel ophaald
        """
        # Ophalen van observatie
        obs = np.zeros((self.grid_size, self.grid_size), dtype=np.int32)
        for x, y in self.snake:
            obs[y][x] = 1
        
        # Ophalen locatie van hoofd
        head_x, head_y = self.snake[0]
        obs[head_y][head_x] = 3
        
        # Ophalen locatie van voedsel
        fx, fy = self.food
        obs[fy][fx] = 2
    
        # Regelen van de timeframe via buffer
        self.obs_buffer.append(obs)
        if len(self.obs_buffer) > 3:
            self.obs_buffer.pop(0)
    
        while len(self.obs_buffer) < 3:
            self.obs_buffer.insert(0, np.zeros((self.grid_size, self.grid_size), dtype=np.int32))
    
        return np.array(self.obs_buffer)

    def step(self, action):
        """
        De method die ervoor zorgt dat de agents getrained worden

        Parameters:
        ----------
            action : int
                De actie die de Agent maakt

        Returns:
        ---------
            observatie, reward, done, truncated, info
        """
        # Indien het spel klaar is
        if self.done:
            return self._get_obs(), 0, True, False, {}

        # Regel om te voorkomen dat de agent 180 graden draait als slang (instant death)
        if abs(action - self.direction) == 2:
            action = self.direction

        # Opnieuw instellen direction voor volgende iteratie
        self.direction = action

        # Kijken waar de snake is t.o.v. de actie
        dx = [0, 1, 0, -1]
        dy = [-1, 0, 1, 0]
        head_x, head_y = self.snake[0]
        new_head = (head_x + dx[action], head_y + dy[action])

        # Indien die zichzelf of de muur raakt is het spel voorbij
        if (new_head in self.snake or
            not 0 <= new_head[0] < self.grid_size or
            not 0 <= new_head[1] < self.grid_size):
            self.done = True
            return self._get_obs(), -10, True, False, {}

        # Langer maken snake
        self.snake.insert(0, new_head)

        # Bepalen afstand tussen snake en appel
        fx, fy = self.food
        old_dist = abs(head_x - fx) + abs(head_y - fy)
        new_dist = abs(new_head[0] - fx) + abs(new_head[1] - fy)

        # Resetten van steps
        self.steps = 0
        self.steps += 1

        # Reward logica
        # 50 punten voor eten appel
        if new_head == self.food:
            reward = 50
            self.food = self._place_food()
            self.steps = 0
        # Reward op basis van afstand naar appel en weghalen laatste lichaamsdeel
        else:
            reward = (old_dist - new_dist) * 0.5 - 0.2
            self.snake.pop()

        # Tegengaan en beëindigen spel bij infinite loops
        if self.steps >= 100:
            reward = -10
            self.done = True
            return self._get_obs(), reward, True, False, {}

        # Tonen van het spel bij render_mode == "human"
        if self.render_mode == "human":
            self.render()

        return self._get_obs(), reward, False, False, {}

    def _init_render(self):
        """
        Initialiser van de Pygame
        """
        pygame.init()
        self.window = pygame.display.set_mode(
            (self.grid_size * self.cell_size, self.grid_size * self.cell_size))
        pygame.display.set_caption("Snake AI")
        self.clock = pygame.time.Clock()

    def render(self):
        """
        Renderen van het spel tijdens het spelen. Deze method
        zorgt ervoor dat de snake en appel getoont worden op het scherm
        en kunnen bewegen over het scherm.
        """
        # Renderen indien dit nog niet is gebeurd
        if self.window is None:
            self._init_render()

        # Tekenen van de snake op het scherm
        self.window.fill((0, 0, 0))
        for x, y in self.snake:
            pygame.draw.rect(
                self.window,
                (0, 255, 0),
                pygame.Rect(x * self.cell_size, y * self.cell_size, self.cell_size, self.cell_size)
            )

        # Tekenen van het voedsel op het scherm
        fx, fy = self.food
        pygame.draw.rect(
            self.window,
            (255, 0, 0),
            pygame.Rect(fx * self.cell_size, fy * self.cell_size, self.cell_size, self.cell_size)
        )
        
        # Highlighten wat de agent kan zien
        if self.snake:
            head_x, head_y = self.snake[0]
            r = self.view_radius
            for dy in range(-r, r+1):
                for dx in range(-r, r+1):
                    nx, ny = head_x + dx, head_y + dy
                    if 0 <= nx < self.grid_size and 0 <= ny < self.grid_size:
                        pygame.draw.rect(
                            self.window,
                            (50, 50, 50),  # blauw = zichtgebied
                            pygame.Rect(nx * self.cell_size, ny * self.cell_size, self.cell_size, self.cell_size),
                            width = 1  # alleen rand tekenen
                        )

        pygame.display.flip()
        self.clock.tick(10)

    def close(self):
        """
        Method die de game goed afsluit om kernal crashes te voorkomen
        """
        if self.window:
            pygame.quit()

## Trainen van het model

In [ ]:
class EvalAndRenderCallback(BaseCallback):
    """
    Class die ervoor zorgt dat je kan zien wat er gebeurt tijdens het trainen
    """
    def __init__(self, eval_env, render_freq=10_000, verbose=0):
        super().__init__(verbose)
        self.eval_env = eval_env
        self.render_freq = render_freq
        self.episodes_run = 0

    def _on_step(self) -> bool:
        if self.num_timesteps % self.render_freq == 0:
            obs, _ = self.eval_env.reset()
            done = False
            while not done:
                action, _ = self.model.predict(obs, deterministic=True)
                obs, reward, done, truncated, info = self.eval_env.step(action)
                self.eval_env.render()
        return True


# Maak een instance van je custom omgeving
env = SnakeEnv(render_mode=None)

# Wrap de omgeving (nodig voor Stable-Baselines3)
vec_env = DummyVecEnv([lambda: SnakeEnv(render_mode=None)])

# Initialiseer PPO
model = PPO(
    "MlpPolicy",             
    vec_env,
    verbose=1,
    learning_rate=2.5e-4,
    n_steps=2048,
    batch_size=64,
    n_epochs=10,
    tensorboard_log="./ppo_snake_tensorboard/",
    device="cpu"
)

# Trainen van het model met callback uit voor RAM bescherming bij grote aantallen
model.learn(total_timesteps=100_000, callback=None)

# Opslaan van het model
model.save("ppo_snake_smallerview_5M_2.5learningrate")

# Om later opnieuw te laden:
# model = PPO.load("ppo_snake")

# Testen van het getrainde model
test_env = SnakeEnv(render_mode="human")
obs, _ = test_env.reset()
done = False

while not done:
    action, _states = model.predict(obs)
    obs, reward, done, truncated, info = test_env.step(action)
    test_env.render()

test_env.close()

## Testen van het model

In [ ]:
# Heraanmaken van environment voor testen
eval_env = SnakeEnv(render_mode="human")

# Laden van het getrainde model
model_path = "ppo_snake_timeframe_10M.zip"
model = PPO.load(model_path, env=eval_env,device='cpu')

# Lijsten en bepalen aantal test episodes
num_episodes = 10
total_rewards = []
food_counts = []
highscore = 0

# Starten met het testen van het model
for episode in range(num_episodes):
    obs, _ = eval_env.reset()
    done = False
    episode_reward = 0
    step_count = 0
    food_eaten = 0
    steps_since_food = 0  

    while not done:
        # Laat het model de acties voorspellen
        action, _ = model.predict(obs, deterministic=True)
        prev_snake_len = len(eval_env.snake)

        obs, reward, done, truncated, info = eval_env.step(action)
        episode_reward += reward
        step_count += 1
        steps_since_food += 1  # tel stappen sinds laatste appel

        # Check of voedsel is gegeten
        if len(eval_env.snake) > prev_snake_len:
            food_eaten += 1
            steps_since_food = 0  # reset bij eten

        # Stop als er 200 stappen geen voedsel is gegeten
        if steps_since_food >= 200:
            print("⚠️  Terminating: no food eaten in 200 steps.")
            break

        time.sleep(0.1)

    # Opslaan gegevens en tonen van de episode statistieken
    total_rewards.append(episode_reward)
    food_counts.append(food_eaten)
    highscore = max(highscore, food_eaten)

    print(f"\n✅ Episode {episode+1} finished.")
    print(f"🔸 Reward: {episode_reward:.2f}")
    print(f"🍎 Food eaten: {food_eaten}")
    print(f"🏆 Highscore so far: {highscore}")

# Na alle episodes
avg_reward = sum(total_rewards) / len(total_rewards)
avg_food = sum(food_counts) / len(food_counts)

print("\n==== Test Summary ====")
print(f"Average reward: {avg_reward:.2f}")
print(f"Average food per episode: {avg_food:.2f}")
print(f"Highscore (most food): {highscore}")